### IMPORTS:

In [ ]:
# SELENIUM:
from selenium import webdriver                                      #driver
from selenium.webdriver.chrome.options import Options               #driver options
from webdriver_manager.chrome import ChromeDriverManager            #driver manager
from selenium.webdriver.common.by import By                         #for driver request
from selenium.webdriver.support.ui import WebDriverWait             #for waiting time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

# MONGO:
import pymongo
from pymongo import MongoClient

# TIEMPOS:
import time                                                           #to add time
from time import sleep

# FECHA
from datetime import datetime

# PERSONAL INFO:
from Passwords import password
from Passwords import email


### Conexión a MongoDB:

In [ ]:
cliente = MongoClient('localhost', 27017)
base_de_datos = cliente['scrap_link']
collection = base_de_datos['scrap_perfiles']                     # colección donde guardamos los datos
collection.create_index([("enlace", pymongo.DESCENDING)]) 

### DRIVER CONECTION:

In [ ]:
PROXY = "IpOfTheProxy:PORT" 
chrome_options = Options() 
ChromeDriverManager().install()                                        #install
driver = webdriver.Chrome() 

# Bot detection:

# Add the proxy as argument 
chrome_options.add_argument("--proxy-server=%s" % PROXY) 


chrome_options.add_argument("--disable-blink-features=AutomationControlled") 
 
# Exclude the collection of enable-automation switches 
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"]) 
 
# Turn-off userAutomationExtension 
chrome_options.add_experimental_option("useAutomationExtension", False) 

# Changing the property of the navigator value for webdriver to undefined 
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

### LOGIN:

In [ ]:
def login(driver, email=None, password=None, cookie = None, timeout=10):
    if cookie is not None:
        return _login_with_cookie(driver, cookie)
  
    if not email or not password:
        email, password = __prompt_email_password()
  
    driver.get("https://www.linkedin.com/login")
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username")))
  
    email_elem = driver.find_element(By.ID,"username")
    email_elem.send_keys(email)
  
    password_elem = driver.find_element(By.ID,"password")
    password_elem.send_keys(password)
    password_elem.submit()
  
    element = WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.CLASS_NAME, c.VERIFY_LOGIN_ID)))

In [ ]:
email = "email"
password = "password"
login(driver, email, password )

### SCRAP PERFILES:

In [ ]:
def scrap(lst_dicc):
    
    try:
        for dicc in lst_dicc:
            
            datos_finales_experiencia = {'experiencias':[]}
            datos_finales_educacion = {'educacion':[]}
            name = dicc['nombre']
            url = dicc['enlace']
            driver.get(url)
            time.sleep(10)
            
            #EXPERIENCIA:
            try:
                experiencias = driver.find_elements(By.CSS_SELECTOR, 'a[data-field=experience_company_logo]')                               # Selecionamos las cajas de las experiencias a través del id del logo
                experiencias_logo = []                                                                                                      # Definimos un objeto donde guardar sólo los objetos de selenium del logo(evitar casos de experiencias complejas que coge dos veces cada compleja)

                for experiencia in experiencias:

                    child_text = experiencia.find_element(By.CSS_SELECTOR, "*").text                                                        # Encuentra todos los elementos del ID anterior que tengan texto

                    if not child_text:                                                                   
                        experiencias_logo.append(experiencia)                                                                               # Guarda aquellos elementos que no tengan texto (los elementos relacionados con los logos de la empresa)


                for logo in experiencias_logo:

                    experiencia = {'nombre_empresa': None, 'link':None,'puestos': []}

                    experiencia["link"] = logo.get_attribute('href')                                                                         # Link de la empresa   

                    # En logos

                    job_info = logo.find_element(By.XPATH, ("../..")).find_element(By.CSS_SELECTOR, "div:nth-child(2)>  div:nth-child(1)")   # Desde los logos, subre a dos padres, baja al hijo segundo y de ahí al hijo primero

                    job_header = job_info.find_element(By.CSS_SELECTOR, "*")

                    tag = job_header.tag_name

                    # Experiencia simple:

                    if tag == "div":

                        puesto = {}
                        try:
                            puesto["nombre_puesto"] = job_header.find_element(By.CSS_SELECTOR, "div:nth-child(1) > div:nth-child(1) > span.visually-hidden").text
                        except:
                            pass
                        try:
                            puesto["fecha_inicio"] = job_header.find_element(By.CSS_SELECTOR, "span:nth-child(3) > span.visually-hidden").text.split(" - ")[0]
                        except:
                            pass
                        try:
                            puesto["fecha_fin"] = job_header.find_element(By.CSS_SELECTOR, "span:nth-child(3) > span.visually-hidden").text.split(" - ")[1].split(" · ")[0]
                            if puesto["fecha_fin"] == "actualidad":
                                puesto["fecha_fin"] = datetime.now().strftime("%Y-%m")
                        except:
                            pass
                        try:
                            puesto["duracion"] = job_header.find_element(By.CSS_SELECTOR, "span:nth-child(3) > span.visually-hidden").text.split(" - ")[1].split(" · ")[1]
                        except:
                            pass
                        try:
                            puesto["ubicacion"] = job_header.find_element(By.CSS_SELECTOR, "span:nth-child(4) > span.visually-hidden").text
                        except:
                            pass
                        try:
                            experiencia["nombre_empresa"] = job_header.find_element(By.CSS_SELECTOR, "span:nth-child(2) > span.visually-hidden").text
                        except:
                            pass
                        try:
                            experiencia["puestos"].append(puesto)
                        except:
                            pass

                    # Experiencia compleja:

                    else:

                        experiencia["nombre_empresa"] = job_header.find_element(By.CSS_SELECTOR, "div:nth-child(1) > div:nth-child(1) > span.visually-hidden").text

                        jobs_list = logo.find_element(By.XPATH, ("../..")).find_element(By.CSS_SELECTOR, "div:nth-child(2) > div:nth-child(2) > ul:nth-child(1)").find_elements(By.CLASS_NAME, "optional-action-target-wrapper.display-flex.flex-column.full-width")

                        for job in jobs_list:

                            puesto = {}
                            try: 
                                puesto["nombre_puesto"] = job.find_element(By.CSS_SELECTOR, "div > span.visually-hidden").text
                            except:
                                pass
                            try:
                                spans = job.find_elements(By.CLASS_NAME, 't-14.t-normal.t-black--light')
                            except:
                                pass
                            try:
                                puesto["fecha_inicio"] = spans[0].find_element(By.CSS_SELECTOR, 'span.visually-hidden').text.split(" - ")[0]
                            except:
                                pass
                            try:
                                puesto["fecha_fin"] = spans[0].find_element(By.CSS_SELECTOR, 'span.visually-hidden').text.split(" - ")[1].split(" · ")[0]
                                if puesto["fecha_fin"] == "actualidad":
                                    puesto["fecha_fin"] = datetime.now().strftime("%Y-%m")
                            except:
                                pass
                            try:
                                puesto["duracion"] = spans[0].find_element(By.CSS_SELECTOR, 'span.visually-hidden').text.split(" - ")[1].split(" · ")[1]
                            except:
                                pass
                            try:
                                puesto["ubicacion"] = spans[1].find_element(By.CSS_SELECTOR, 'span.visually-hidden').text
                            except:
                                pass


                            experiencia["puestos"].append(puesto)



                    datos_finales_experiencia['experiencias'].append(experiencia)

                    # To MONGODB
                try:
                    collection.insert_one({'enlace': url, 'nombre': name, 'experiencia_laboral' : datos_finales_experiencia})

                except pymongo.errors.DuplicateKeyError as e:
                    pass
                
            except Exception as e:
                print(f"No se pudo encontrar o scrapear 'experience' para {nombre}")
            time.sleep(1)
                
            #EDUCACIÓN:
            try:
                tarjetas = driver.find_element(By.ID, 'education')
                tarjetas_estudios = tarjetas.find_element(By.XPATH, ("..")).find_elements(By.CSS_SELECTOR, "ul >li > div > div > div > a")


                educacion = {'institucion': None, 'carrera':None,'fecha_inicio': None, 'fecha_fin': None}
                for tarjetas_estudio in tarjetas_estudios:

                    estudios = {}
                    try:
                        estudios['institucion'] = tarjetas_estudio.find_element(By.CSS_SELECTOR, 'span.visually-hidden').text
                    except:
                        pass
                    try:
                        estudios['carrera'] = tarjetas_estudio.find_element(By.CSS_SELECTOR, "span:nth-child(2) > span:nth-child(2)").text
                    except:
                        pass
                    try:
                        estudios['fecha_inicio'] = tarjetas_estudio.find_element(By.CSS_SELECTOR, "span.t-14.t-normal.t-black--light > span:nth-child(2)").text.split(" - ")[0]
                    except:
                        pass
                    try:
                        estudios['fecha_fin'] = tarjetas_estudio.find_element(By.CSS_SELECTOR, "span.t-14.t-normal.t-black--light > span:nth-child(2)").text.split(" - ")[1]
                    except:
                        pass
                    
                    datos_finales_educacion['educacion'].append(estudios)



                # To MONGODB
                try:
                    collection.insert_one({'enlace': url, 'nombre': name, 'educación' : datos_finales_educacion})

                except pymongo.errors.DuplicateKeyError as e:
                    pass

            except Exception as e:
                print(f"No se pudo encontrar o scrapear 'education' para {nombre}")
                
    except Exception as e:
        print(f"Ocurrió un error: {str(e)}")
        pass
    
    
    #finally:
        
    #    driver.quit()


In [ ]:
# Accedemos a los datos del anterior scrapeo:

db = cliente['scrap_link']                                               # db con los datos juntos
collection_1 = db['link_name_final']                                     # coleccion con los datos juntos

# Obtener todos los documentos de la colección
documents = collection_1.find()

# Crear una lista de diccionarios con solo las dos claves requeridas
lst_dicc = [{'nombre': doc.get('nombre'), 'enlace': doc.get('enlace')} for doc in documents]

len(lst_dicc)

In [ ]:
scrap(lst_dicc)